In [ ]:
import pandas as pd
import numpy as np
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/movie_data/'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/movie_data/movie.csv
/content/drive/MyDrive/movie_data/link.csv
/content/drive/MyDrive/movie_data/genome_scores.csv
/content/drive/MyDrive/movie_data/genome_tags.csv
/content/drive/MyDrive/movie_data/rating.csv
/content/drive/MyDrive/movie_data/tag.csv
/content/drive/MyDrive/movie_data/NaverToonInfo.xlsx
/content/drive/MyDrive/movie_data/NaverToonInfo.csv


In [ ]:
movie=pd.read_csv('/content/drive/MyDrive/movie_data/movie.csv')
rating=pd.read_csv('/content/drive/MyDrive/movie_data/rating.csv')

Item-Item Collaborative Filtering

 Cosine distance의 Weighted average

In [ ]:
#그대로 user movie matirx를 만들기에는 데이터의 크기가 너무 크기 때문에
#500개 이상 평가한 user들만 filtering
x=rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating=rating[rating['userId'].isin(x[x].index)]
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

#마찬가지로 movie도 rating 수로 filtering
y=thin_rating['movieId'].value_counts()>500
print(len(y[y]))
thin_rating=thin_rating[thin_rating['movieId'].isin(y[y].index)]
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

7441
3124


In [ ]:
#NaN 결측치를 일단 0으로 채우기
movie_pivot.fillna(0,inplace=True)
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,4.0,0.0,3.0,4.0,5.0,4.0,...,0.0,2.5,3.5,5.0,3.0,0.0,4.0,5.0,4.0,5.0
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
5,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#이 Item과 비슷하게 user들에게 평가받은 item을 cosine similarity wieght로 찾음
item_item_collabor = cosine_similarity(movie_pivot.loc[movie_pivot.index==2],movie_pivot)
item_item_collabor

array([[0.74390008, 1.        , 0.4895084 , ..., 0.24016086, 0.20521981,
        0.25998092]])

In [ ]:
data_coll = []
for i in range(len(item_item_collabor[0])):
  temp = item_item_collabor[0]
  data_coll.append(temp[i])

In [ ]:
#유사한 상위 10개의 영화
abc = pd.DataFrame(data=data_coll, columns = ['user24'], index = movie_pivot.index)
abc['user24'].sort_values(ascending=False)[:10]

movieId
2       1.000000
480     0.773736
780     0.762028
1580    0.760161
356     0.758653
364     0.755307
367     0.751001
2571    0.747856
1270    0.746507
586     0.744588
Name: user24, dtype: float64

In [ ]:
a= abc['user24'].sort_values(ascending=False)[:10].index
list(map(str, a))

['2', '480', '780', '1580', '356', '364', '367', '2571', '1270', '586']

In [ ]:
values = movie_pivot.loc[a,24:24][:10].values
weight = abc['user24'].sort_values(ascending=False)[:10].values

In [ ]:
# cosine similarity wieght를 부여한 User24의 movie 2에 대한 예상 평점
cosine_weight_sum = []
weight_sum = 0
for i in range(10):
  if values[i] != 0:
    weight_sum += weight[i]
    cosine_weight_sum.append(values[i]*weight[i])
cosine_weight_avg = sum(cosine_weight_sum)/weight_sum
cosine_weight_avg

array([4.28168884])

In [ ]:
movie_pivot.loc[a,:]

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
480,5.0,3.0,5.0,4.0,3.5,0.0,4.0,4.0,5.0,4.5,...,4.0,3.0,3.5,5.0,5.0,4.5,4.0,4.0,0.0,5.0
780,5.0,4.0,5.0,2.0,3.0,0.0,1.0,3.0,5.0,4.0,...,3.0,1.5,3.0,3.0,5.0,5.0,3.0,2.5,4.5,0.0
1580,5.0,5.0,4.0,4.0,3.5,3.0,2.5,4.0,5.0,1.5,...,3.0,2.0,3.0,4.5,5.0,5.0,3.5,4.0,4.5,5.0
356,5.0,5.0,5.0,4.5,4.0,0.0,4.0,3.0,5.0,1.0,...,4.0,3.5,0.0,4.5,5.0,4.5,4.0,4.0,0.0,5.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
2571,5.0,5.0,5.0,5.0,3.5,1.0,4.0,3.0,5.0,3.0,...,3.0,3.5,4.0,4.0,5.0,4.5,4.0,5.0,5.0,5.0
1270,5.0,4.0,5.0,0.0,4.0,3.0,3.5,3.0,4.0,4.0,...,4.0,3.5,3.5,5.0,4.0,4.5,4.0,4.5,4.0,0.0


In [ ]:
(0.773736*3 + 0.762028*4 + 0.760161*5+0.758653*5+0.751001*4+0.747856*5+0.746507*4)/(0.773736+0.762028+0.760161+0.758653+0.751001+0.747856+0.746507)

4.2816887430088855

Pearson correlation 의 Weighted average


In [ ]:
pearson = np.corrcoef(movie_pivot.loc[movie_pivot.index==2],movie_pivot)
pearson

array([[1.        , 0.23028695, 1.        , ..., 0.03351904, 0.00821129,
        0.05194667],
       [0.23028695, 1.        , 0.23028695, ..., 0.02054619, 0.00389995,
        0.03664232],
       [1.        , 0.23028695, 1.        , ..., 0.03351904, 0.00821129,
        0.05194667],
       ...,
       [0.03351904, 0.02054619, 0.03351904, ..., 1.        , 0.57148563,
        0.67643725],
       [0.00821129, 0.00389995, 0.00821129, ..., 0.57148563, 1.        ,
        0.53253697],
       [0.05194667, 0.03664232, 0.05194667, ..., 0.67643725, 0.53253697,
        1.        ]])

In [ ]:
data_coll3 = []
for i in range(len(pearson[0])):
  temp = pearson[0]
  if i == 0:
    continue
  data_coll3.append(temp[i])

In [ ]:
item_item_pearson = pd.DataFrame(data=data_coll3, index = movie_pivot.index, columns = ['user24'])
item_item_pearson['user24'].sort_values(ascending=False)[:10]

movieId
2       1.000000
158     0.360144
364     0.350876
3489    0.343065
2617    0.342057
586     0.341812
367     0.337084
317     0.335058
736     0.332709
2054    0.332002
Name: user24, dtype: float64

In [ ]:
c= item_item_pearson['user24'].sort_values(ascending=False)[:10].index
list(map(str, c))

['2', '158', '364', '3489', '2617', '586', '367', '317', '736', '2054']

In [ ]:
values = movie_pivot.loc[c,24:24][:10].values
weight = item_item_pearson['user24'].sort_values(ascending=False)[:10].values

In [ ]:
# pearson similarity wieght를 부여한 User24의 movie 2에 대한 예상 평점
cosine_weight_sum = []
weight_sum = 0
for i in range(10):
  if values[i] != 0:
    weight_sum += weight[i]
    cosine_weight_sum.append(values[i]*weight[i])
cosine_weight_avg = sum(cosine_weight_sum)/weight_sum
cosine_weight_avg

array([4.])

In [ ]:
movie_pivot.loc[c,:]

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
158,4.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
3489,0.0,0.0,2.0,1.0,3.5,0.0,2.5,0.0,3.0,0.0,...,3.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
2617,0.0,0.0,4.0,4.0,3.0,2.0,2.5,3.0,5.0,0.0,...,0.0,1.0,3.0,2.5,5.0,0.0,0.0,2.5,4.0,0.0
586,4.0,0.0,3.0,1.0,3.0,0.0,1.0,0.0,4.0,3.5,...,2.0,3.0,0.0,3.5,3.0,0.0,4.0,0.0,0.0,4.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
317,0.0,4.0,0.0,3.5,3.5,0.0,1.5,0.0,4.0,0.0,...,0.0,0.0,0.0,3.0,5.0,0.0,0.0,3.5,0.0,5.0
736,5.0,0.0,4.0,4.0,3.0,0.0,1.0,0.0,4.0,2.5,...,1.0,1.0,3.5,3.0,5.0,0.0,0.0,2.0,0.0,0.0


In [ ]:
(4*0.337084 + 4*0.335058)/(0.337084+0.335058)

4.0

 User-User Collaborative Filtering

In [ ]:
#그대로 user movie matirx를 만들기에는 데이터의 크기가 너무 크기 때문에
#500개 이상 평가한 user들만 filtering
x=rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating=rating[rating['userId'].isin(x[x].index)]
movie_pivot=thin_rating.pivot_table(columns='movieId',index='userId',values='rating')

#마찬가지로 movie도 rating 수로 filtering
y=thin_rating['movieId'].value_counts()>50
print(len(y[y]))
thin_rating=thin_rating[thin_rating['movieId'].isin(y[y].index)]
movie_pivot=thin_rating.pivot_table(columns='movieId',index='userId',values='rating')

7441
9313


In [ ]:
#NaN 결측치를 일단 0으로 채우기
movie_pivot.fillna(0,inplace=True)
movie_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,116887,116977,117176,117590,118696,118700,118900,118997,119141,119145
userId,,,,,,,,,,,,,,,,,,,,,
11,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,4.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,4.0,3.5,3.0,0.0,0.0,0.0,2.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138397,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138406,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138411,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_pivot[1:2]

movieId,1,2,3,4,5,6,7,8,9,10,...,116887,116977,117176,117590,118696,118700,118900,118997,119141,119145
userId,,,,,,,,,,,,,,,,,,,,,
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pearson = np.corrcoef(movie_pivot[1:2],movie_pivot)
pearson

array([[1.        , 0.20047622, 1.        , ..., 0.38559683, 0.21073375,
        0.33880983],
       [0.20047622, 1.        , 0.20047622, ..., 0.24495053, 0.27071944,
        0.19438588],
       [1.        , 0.20047622, 1.        , ..., 0.38559683, 0.21073375,
        0.33880983],
       ...,
       [0.38559683, 0.24495053, 0.38559683, ..., 1.        , 0.29312898,
        0.33824489],
       [0.21073375, 0.27071944, 0.21073375, ..., 0.29312898, 1.        ,
        0.15136137],
       [0.33880983, 0.19438588, 0.33880983, ..., 0.33824489, 0.15136137,
        1.        ]])

In [ ]:
data_coll4 = []
for i in range(len(pearson[0])):
  temp = pearson[0]
  if i ==0:
    continue
  data_coll4.append(temp[i])

In [ ]:
item_item_pearson = pd.DataFrame(data=data_coll4, index = movie_pivot.index, columns = ['movie2'])
item_item_pearson['movie2'].sort_values(ascending=False)[:10]

userId
24        1.000000
96551     0.568794
54745     0.519707
43849     0.502208
93580     0.494508
298       0.493165
110670    0.493084
47195     0.487736
75079     0.487634
84269     0.486360
Name: movie2, dtype: float64

In [ ]:
d= item_item_pearson['movie2'].sort_values(ascending=False)[:10].index
list(map(str, d))

['24',
 '96551',
 '54745',
 '43849',
 '93580',
 '298',
 '110670',
 '47195',
 '75079',
 '84269']

In [ ]:
movie_pivot.loc[d,2:2].replace(0, np.nan).mean(skipna=True)

movieId
2    3.25
dtype: float64

In [ ]:
values = movie_pivot.loc[d,2:2][:10].values
weight = item_item_pearson['movie2'].sort_values(ascending=False)[:10].values

In [ ]:
pearson_weight_sum = []
weight_sum = 0
for i in range(10):
  if values[i] != 0:
    weight_sum += weight[i]
    pearson_weight_sum.append(values[i]*weight[i])
pearson_weight_avg = sum(pearson_weight_sum)/weight_sum
pearson_weight_avg

array([3.25006536])